In [35]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity


import numpy as np
import re



In [36]:
def clean_filename(url):
    return re.sub(r'[\/:*?"<>|]', '_', url)

In [37]:
url_list = [
    "https://edition.cnn.com/2024/05/13/tech/australia-us-undersea-drones-ml-hnk-intl/index.html",
    "https://www.bbc.com/news/articles/cn7l375z720o", 
    "https://www.bbc.com/news/articles/c6240qepyppo"
]
for url in url_list:
    response = requests.get(url)
    if response.status_code == 200:
        html_content = response.text  
    else:
        print(f"Failed to retrieve {url}")
    html_content  
    soup = BeautifulSoup(html_content, 'html.parser')
    paragraphs = soup.find_all('p')
    article_text = ' '.join([para.get_text() for para in paragraphs])
    print(article_text)

    with open(f"scraped_article_{clean_filename(url)}.txt", "w") as f:
        f.write(article_text)

    nltk.download('punkt')
    nltk.download('stopwords')

    with open("scraped_article.txt", "r") as f:
        article_text = f.read()

    sentences = sent_tokenize(article_text)

    stop_words = set(stopwords.words('english'))
    cleaned_sentences_nltk = []

    for sentence in sentences:
        words = word_tokenize(sentence.lower())
        words = [word for word in words if word not in stop_words and word not in string.punctuation]
        cleaned_sentences_nltk.append(' '.join(words))

    print(cleaned_sentences_nltk)
    cvect = CountVectorizer(ngram_range=(1,1), token_pattern='(?u)\\b\\w+\\b')
    counts = cvect.fit_transform(cleaned_sentences_nltk)
    normalized_counts = normalize(counts, norm='l1', axis=1)

    tfidf = TfidfVectorizer(ngram_range=(1,1))
    tfs = tfidf.fit_transform(cleaned_sentences_nltk)

    sentence_scores = np.sum(tfs.toarray(), axis=1)

    num_sentences = 5
    top_sentence_indices = np.argsort(sentence_scores)[-num_sentences:]

    # Sélection des phrases les plus importantes
    summary = [sentences[i] for i in sorted(top_sentence_indices)]

    # Affichage du résumé extractif
    print("Résumé extractif:")
    for sentence in summary:
        print(sentence)
    
    vectorizer = CountVectorizer().fit_transform(summary)
    vectors = vectorizer.toarray()
    cosine_matrix = cosine_similarity(vectors)

    threshold = 0.7
    redundant_sentences = set()

    for i in range(len(cosine_matrix)):
        for j in range(i + 1, len(cosine_matrix)):
            if cosine_matrix[i][j] > threshold:
                redundant_sentences.add(j)


    filtered_summary = [sentence for idx, sentence in enumerate(summary) if idx not in redundant_sentences]
    
    with open(f"summary_{clean_filename(url)}.txt", "w") as f:
        for sentence in filtered_summary:
            f.write(sentence + "\n")
        f.close()

Markets 


 Hot Stocks 


 Fear & Greed Index 


 
            Latest Market News 


 
            Hot Stocks 


 
            Ghost Shark and Manta Ray protect the undersea realm. Sounds like the plot of a future Marvel movie, but in actual fact, it’s what could be the future of Pacific naval defenses.
     
            Ghost Shark and Manta Ray are the names of prototype uncrewed underwater vehicles – UUVs or drones – introduced recently by Australia and the United States respectively.
     
            Experts say the submersibles could represent the future of undersea warfare, showing the ability to exert power while minimizing the danger to human life.
     
            The use of drones in aerial warfare has become commonplace. The US used them extensively during conflicts in Iraq and Afghanistan beginning in the 1990s, and newer, cheaper drones have become key pieces of military hardware for both sides in Russia’s invasion of Ukraine.
     
            Kyiv has also built naval 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Western technology and finance are helping Ukraine carry out hundreds of long-range strikes inside Russia. That is despite Nato allies still refusing to give Ukraine permission to use Western-supplied munitions to do so – mostly because of fears of escalation. Ukraine has been stepping up its long-range strikes inside Russia over the past few months, launching scores of drones simultaneously at strategic targets several times a week.  The targets include air force bases, oil and ammunition depots and command centres. Ukrainian firms are now producing hundreds of armed one-way attack drones a month, at a fraction of the cost it takes to produce a similar drone in the West.  One company told the BBC it was already creating a disproportionate impact on Russia’s war economy at a relatively small expense. The BBC has been briefed by a number of those involved in these missions. They include one of Ukraine’s largest one-way attack drone manufacturers, as well as a big data company which has 